## 0. Import Libraries

In [2]:
import pandas as pd

### 0.1 Load Dataset and Format

In [34]:
#import dataframe
data = pd.read_csv(r'FF_NN_50m_MaxAbs_Undersample_results.csv')
data['calipso_datetime'] = pd.to_datetime(data['calipso_datetime'])
data['calip_lat'] = pd.to_numeric(data['calip_lat'])
data['calip_lon'] = pd.to_numeric(data['calip_lon'])
data['crw_baa_7d_max'] = pd.to_numeric(data['crw_baa_7d_max'])
data['crw_baa_7d_max'] = data['crw_baa_7d_max'] - 1

data.loc[(data['y_pred'] == 1), 'y_pred'] = "Coral"
data.loc[(data['y_pred'] == 0), 'y_pred'] = 'Other'

## 1.0 Define html Formatting Functions

In [39]:
#define functions to apply legend to map and convert df to json format
def add_categorical_legend(folium_map, title, colors, labels):
    import folium
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

def create_geojson_features(df):
    print('> Creating GeoJSON features...')
    features = []
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['calip_lon'],row['calip_lat']]
            },
            'properties': {
                'time': row['calipso_datetime'].date().__str__(),
                'popup': str('Label: {}, Date: {}, Coordinates: {}, Bleaching Alert: {}').format(row['y_pred'],
                                                                                          str(row['calipso_datetime'].date()), 
                                                                                          [row['calip_lat'], row['calip_lon']],
                                                                                          row['crw_baa_7d_max']),
                'style': {'color' : row['color']},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': row['color'],
                    'fillOpacity': 1,
                    'stroke': 'true',
                    'radius': 7
                }
            }
        }
        features.append(feature)
    return features

### 1.2 Define Function to Create Map and Extract Features from the Dataset

In [40]:
def create_map(df):
    import folium
    import numpy as np
    import pandas as pd
    from folium.plugins import TimestampedGeoJson
    
    #filter for relevant variables
    filtered_df = df[['calipso_datetime', 'calip_lat', 'calip_lon', 'crw_baa_7d_max', 'y_pred']]

    #define colors based off class
    colors = []
    for index, row in filtered_df.iterrows():
        if row['y_pred'] == 'Coral':
            if row['crw_baa_7d_max'] > 0:
                colors.append('#FFC300')
            else:
                colors.append('#a1d996')
        else:
            colors.append('#000')

    filtered_df['color'] = colors

    #use list comprehension to format date column
    filtered_df['calipso_datetime'] = [row.strftime("%Y-%m-%d") for row in filtered_df['calipso_datetime']]
    filtered_df['calipso_datetime'] = pd.to_datetime(filtered_df['calipso_datetime'])

    features = create_geojson_features(filtered_df)
    import folium
    from folium.plugins import TimestampedGeoJson
    print('> Making map...')
    coords=[25.0115, -80.5151]
    m = folium.Map(location=coords, control_scale=True, zoom_start=8)

    #create the map with json features
    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features},
        period='P1M',
        duration='P1M',
        transition_time=800,
        auto_play=True).add_to(m)

    m = add_categorical_legend(m, 'Legend',
                                colors = ['#000','#a1d996', '#FFC300'],
                                labels = ['Other', 'Coral', 'Coral w/ Bleaching Conditions'])
    
    #save html to active directory, it crashes my notebook if I run it within the instance
    m.save('timeseries.html')
    return('> Map saved to local drive')
    #return(m)

## 3.0 Generate Map

In [41]:
create_map(data)

C:\Users\quinn\AppData\Local\Temp/ipykernel_17412/3901970195.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['color'] = colors
C:\Users\quinn\AppData\Local\Temp/ipykernel_17412/3901970195.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['calipso_datetime'] = [row.strftime("%Y-%m-%d") for row in filtered_df['calipso_datetime']]
C:\Users\quinn\AppData\Local\Temp/ipykernel_17412/3901970195.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

> Creating GeoJSON features...
> Making map...


'> Map saved to local drive'